In [13]:
import uuid
from pydantic import BaseModel, Field
from langchain_groq import ChatGroq
from langgraph.graph import START, END, StateGraph
from langchain_core.messages import HumanMessage, BaseMessage, SystemMessage, AIMessage
from langgraph.graph.message import add_messages
from langgraph.store.base import BaseStore
from langchain_core.runnables import RunnableConfig
from dotenv import load_dotenv
from typing import TypedDict, Annotated, List
from langgraph.store.memory import InMemoryStore
from CONFIG import GROQ_MODEL, TEMPERATURE

In [14]:
store = InMemoryStore()

In [15]:
load_dotenv()
llm = ChatGroq(model=GROQ_MODEL, temperature=TEMPERATURE)

In [16]:
class pydantic_class(BaseModel):
    should_store: bool = Field(description="whether to store any memory")
    list_of_memory : List[str] = Field(default_factory=List, description="atomic user memory to store")

structured_llm = llm.with_structured_output(pydantic_class)

In [17]:
class state_class(TypedDict):
    messages: Annotated[list[BaseMessage], add_messages]

In [18]:
def stored_LTM_memory(state: state_class, config: RunnableConfig, store: BaseStore):
    config = config['configurable']['user_id']
    namespace = ('user', config, 'details')
    last_msg = state['messages'][-1].content

    decision: pydantic_class = structured_llm.invoke(
        [
            SystemMessage(
                content=(
				"Extract LONG-TERM memories from the user's message.\n"
				"Only store stable, user-specific info (identity, preferences, ongoing projects).\n"
				"Do NOT store transient info.\n"
				"Return should_write=false if nothing is worth storing.\n"
				"Each memory should be a short atomic sentence."
                )
		),
        {'role': 'user', 'content': last_msg}
	  ]
    )


    if decision.should_store:
        for mm in decision.list_of_memory:
            store.put(namespace, str(uuid.uuid4()), {'date': mm})
            
    return {'messages': [{'role': 'assistant', 'content': 'Noted'}]}

In [19]:
builder = StateGraph(state_class)

builder.add_node('stored_LTM_memory', stored_LTM_memory)

builder.add_edge(START, 'stored_LTM_memory')
builder.add_edge('stored_LTM_memory', END)

graph = builder.compile(store=store)

In [20]:
config = {'configurable': {'user_id': 'o1'}}
output = graph.invoke({'messages': [{'role': 'user', 'content': 'Hi my name is adnan saeed'}]},config)
print(output['messages'][-1].content)

Noted


In [21]:
config = {'configurable': {'user_id': 'o1'}}
output = graph.invoke({'messages': [{'role': 'user', 'content': 'i am working as AI Engineer'}]},config)
print(output['messages'][-1].content)

Noted


In [22]:
for i in store.search(('user', 'o1', 'details')):
    print(i.value)

{'date': 'My name is Adnan Saeed'}
{'date': 'The user is working as an AI Engineer'}


In [26]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_openai import ChatOpenAI
from CONFIG import GEMINI_MODEL

In [38]:
openai_llm = ChatOpenAI(model="gpt-5.2-pro", temperature=TEMPERATURE)
gemini_llm = ChatGoogleGenerativeAI(model=GEMINI_MODEL, temperature=TEMPERATURE)
groq_llm = llm
print(f"I'm the GPT:- {openai_llm}")
print(f"I'm the GEMINI:- {gemini_llm}")
print(f"I'm the GROQ:- {groq_llm}")

I'm the GPT:- profile={'max_input_tokens': 272000, 'max_output_tokens': 128000, 'image_inputs': True, 'audio_inputs': False, 'video_inputs': False, 'image_outputs': False, 'audio_outputs': False, 'video_outputs': False, 'reasoning_output': True, 'tool_calling': True, 'structured_output': True, 'image_url_inputs': True, 'pdf_inputs': True, 'pdf_tool_message': True, 'image_tool_message': True, 'tool_choice': True} client=<openai.resources.chat.completions.completions.Completions object at 0x000001E4A0C4C810> async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x000001E4A0C4D410> root_client=<openai.OpenAI object at 0x000001E4A0C4F010> root_async_client=<openai.AsyncOpenAI object at 0x000001E4A0C4C310> model_name='gpt-5.2-pro' model_kwargs={} openai_api_key=SecretStr('**********') stream_usage=True
I'm the GEMINI:- profile={'max_input_tokens': 1048576, 'max_output_tokens': 65536, 'image_inputs': True, 'audio_inputs': True, 'pdf_inputs': True, 'video_inpu

In [34]:
1048576/1000

1048.576

In [37]:
print(f"gpt-4o-mini:- {128000/1000}k")
print(f"gpt-5.2-pro:- {272000/1000}k")

gpt-4o-mini:- 128.0k
gpt-5.2-pro:- 272.0k


In [33]:
131072/1000

131.072